## Intrinsic Dimension : Proof of Concept

This just demonstrates that the Intrinsic Dimension idea works for a simple MNIST-CNN model.

Of possible interest : This PyTorch version is 'generic' in the sense that it should be able
to take arbitrary parameterised models and investigate their Intrinsic Dimension (without
writing specialised `Module_IntrinsicDimension` classes for each module type).

In [3]:
import sys
sys.path.append("/home/jevjev/Dropbox/Projects/Intrinsic-dimensionality/")

In [4]:
import os
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
os.environ["CUDA_VISIBLE_DEVICES"] = str(1)
from intrinsic.utils.general import get_loaders, parameter_count
from models.toy import RegularCNNModel
from models.train_funcs import get_stats_for

In [5]:
mnist_root = '/home/jevjev/Dropbox/Projects/datasets'
n_epochs = 20
batch_size, batch_log_interval, lr = 32, 600, 0.01
seed = 10
try_cuda = True

In [6]:
use_cuda = try_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")

In [7]:
train_loader, test_loader = get_loaders(mnist_root, use_cuda, batch_size)

In [ ]:
get_stats_for(RegularCNNModel(), n_epochs, train_loader, test_loader, batch_log_interval, lr, device)

Total model parameters : 45360
Train Epoch: 1 [     0/ 60000 ( 0%)]	Loss: 2.3573


# Define Intrinsic Dimension Wrapper

In [22]:
class IntrinsicDimensionWrapper(torch.nn.Module):
    
    def __init__(self, module, intrinsic_dimension, verbose=False):
        """
        Wrapper to estimate the intrinsic dimensionality of the 
        objective landscape for a specific task given a specific model
        :param module: pytorch nn.Module
        :param intrinsic_dimension: dimensionality within which we search for solution
        :param verbose: if things should be printed out
        """
        super(IntrinsicDimensionWrapper, self).__init__()
        
        self.verbose = verbose
        
        self.m = [module] # Hide this from inspection by get_parameters()
        
        self.name_base_localname = []
        
        # Stores the initial value: \theta_{0}^{D}
        self.initial_value = dict()
        
        # Stores the randomly generated projection matrix P
        self.random_matrix = dict()
       
        # Parameter vector that is updated, initialised with zeros as per text: \theta^{d}
        V = torch.nn.Parameter( torch.zeros( (intrinsic_dimension) ).to(device) )
        self.register_parameter('V', V)
        v_size = (intrinsic_dimension, )
        
        # Iterates over layers in the Neural Network
        for name, param in module.named_parameters():
            # If the parameter requires gradient update
            if param.requires_grad:
                
                if self.verbose: print(name, param.data.size(), v_size)
                
                # Saves the initial values of the initialised parameters from param.data and sets them to no grad.
                # (initial values are the 'origin' of the search)
                self.initial_value[name] = v0 = param.clone().detach().requires_grad_(False).to(device)
                print(name, v0.size())
                print(v0)
                # If v0.size() is [4, 3], then below operation makes it [4, 3, v_size]
                matrix_size = v0.size() + v_size

                # Generates random projection matrices P, sets them to no grad
                self.random_matrix[name] = (torch.randn(matrix_size, requires_grad=False).to(device) / intrinsic_dimension**0.5)
                
                # NOTE!: lines below are not clear! 
                base, localname = module, name
                while '.' in localname:
                    if self.verbose: print('Local name', localname)
                    prefix, localname = localname.split('.', 1)
                    if self.verbose: print('Prefix', prefix, '  Name', name, '  Local name', localname)
                    base = base.__getattr__(prefix)
                self.name_base_localname.append( (name, base, localname) )

        for name, base, localname in self.name_base_localname:
            delattr(base, localname)

    def forward(self, x):   
        # Iterate over the layers
        for name, base, localname in self.name_base_localname:
            #if self.verbose: print(name, base, localname)
            #print(self.initial_value[name].size(), self.random_matrix[name].size(), self.V.size(), 
            #      torch.matmul(self.random_matrix[name], self.V).size())
            
            # Product between matrix P and \theta^{d}
            ray = torch.matmul(self.random_matrix[name], self.V)
            # Add the \theta_{0}^{D} to P \dot \theta^{d}
            param = self.initial_value[name] + torch.squeeze(ray, -1)

            setattr(base, localname, param)

        # Pass through the model, by getting the module from a list self.m
        module = self.m[0]
        x = module(x)
        return x

# Testing

In [23]:
#model_single= torch.nn.Linear(3,4)
model_single = IntrinsicDimensionWrapper(module=torch.nn.Linear(3, 4), 
                                         intrinsic_dimension=10,
                                         verbose=False)

#[p[0] for p in model_test.named_parameters()]
[ (p.view(-1), p.requires_grad) for p in model_single.parameters() ]
#model_test.initial_value

weight torch.Size([4, 3])
tensor([[ 0.3699,  0.4400,  0.5121],
        [-0.4485, -0.1343,  0.0431],
        [-0.4591,  0.5401,  0.4066],
        [-0.1691,  0.0712, -0.0065]])
bias torch.Size([4])
tensor([ 0.1371, -0.4178, -0.0163, -0.0169])


[(tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], grad_fn=<ViewBackward>),
  True)]

In [25]:
for name, base, localname in model_single.name_base_localname:
    print(model_single.random_matrix[name].size())
    print(model_single.V.size())
    print(torch.matmul(model_single.random_matrix[name], model_single.V))
    print('\n')

torch.Size([4, 3, 10])
torch.Size([10])
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], grad_fn=<UnsafeViewBackward>)


torch.Size([4, 10])
torch.Size([10])
tensor([0., 0., 0., 0.], grad_fn=<MvBackward>)




In [ ]:
x = torch.tensor( [1., 5., -1.25] ).to(device)

model_single(x)

In [ ]:
#model_test.V.data[3]=0.
model_single.V.data[3]+=.005
model_single(x)

In [ ]:
opt_test = torch.optim.Adam(model_single.parameters(), lr=lr)

model_single.train()

#data, target = data.to(device), target.to(device)
x_batch = torch.unsqueeze(x,0)
data, target = x_batch, torch.tensor( [1,] ).to(device)

opt_test.zero_grad()
output = model_single(data)
loss = F.nll_loss(output, target)
print(loss)
loss.backward()
opt_test.step()

model_single.V.requires_grad,
#model_single.m[0].weight.grad, model_single.m[0].weight.grad

# Apply to regular CNN

In [ ]:
## Now, let's build the CNN model with Intrinsic Dimension Wrapping...

intrinsic_dimension_guess = 100

In [ ]:
model_base = RegularCNNModel()
#[name for name,param in model_base.named_parameters()]

model_wrapped = IntrinsicDimensionWrapper( model_base, intrinsic_dimension_guess )
# [name for name,param in model_wrapped.named_parameters()]
#[param for param in model_wrapped.parameters()]

In [ ]:
get_stats_for(model_wrapped)

# Toy MNIST

In [6]:
class FCNAsInPAper(torch.nn.Module):
    def __init__(self):
        super(FCNAsInPAper, self).__init__()
        self.fc1 = torch.nn.Linear(784, 200)
        self.fc2 = torch.nn.Linear(200, 200)
        self.fc3 = torch.nn.Linear(200, 10)

    def forward(self, x):
        
        x = x.view(-1, 784)
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        
        return F.log_softmax(x, dim=1)

In [7]:
intrinsic_dimension_guess = 750

In [8]:
model_base = FCNAsInPAper()

In [12]:
print(parameter_count(model_base))

199210


In [13]:
model_wrapped = IntrinsicDimensionWrapper( model_base, intrinsic_dimension_guess )

In [16]:
get_stats_for(model_wrapped)

Total model parameters : 750
Train Epoch: 1 [     0/ 60000 ( 0%)]	Loss: 2.3145
Train Epoch: 1 [ 19200/ 60000 (32%)]	Loss: 1.0000
Train Epoch: 1 [ 38400/ 60000 (64%)]	Loss: 0.6561
Train Epoch: 1 [ 57600/ 60000 (96%)]	Loss: 0.6059


/home/jevjev/anaconda3/envs/dl/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.5160, Accuracy: 8422/10000 (84.2%)

Train Epoch: 2 [     0/ 60000 ( 0%)]	Loss: 0.8945
Train Epoch: 2 [ 19200/ 60000 (32%)]	Loss: 0.4821
Train Epoch: 2 [ 38400/ 60000 (64%)]	Loss: 0.7526
Train Epoch: 2 [ 57600/ 60000 (96%)]	Loss: 0.8867

Test set: Average loss: 0.6030, Accuracy: 8259/10000 (82.6%)

Train Epoch: 3 [     0/ 60000 ( 0%)]	Loss: 0.4094
Train Epoch: 3 [ 19200/ 60000 (32%)]	Loss: 0.3169
Train Epoch: 3 [ 38400/ 60000 (64%)]	Loss: 1.2058
Train Epoch: 3 [ 57600/ 60000 (96%)]	Loss: 0.8690

Test set: Average loss: 0.4815, Accuracy: 8491/10000 (84.9%)

Train Epoch: 4 [     0/ 60000 ( 0%)]	Loss: 0.8339


KeyboardInterrupt: 